<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/Tensorflow_Decision_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Decsion Forests
- 출처: https://www.tensorflow.org/decision_forests/tutorials/beginner_colab?hl=ko#installing_tensorflow_decision_forests

## Decision Forest
- 지도 분류, 회귀 및 순위 지정을 위한 대규모 기계 학습 알고리즘 제품군

## 01. Tensorflow Decision Forests 설치

In [1]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 11.1 MB/s 
     |████████████████████████████████| 511.7 MB 6.7 kB/s 
     |████████████████████████████████| 438 kB 70.2 MB/s 
     |████████████████████████████████| 5.8 MB 52.3 MB/s 
     |████████████████████████████████| 1.6 MB 54.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unin

In [2]:
!pip install wurlitzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 02. 라이브러리 불러오기

In [3]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [4]:
# 출력 높이 제한
@register_line_magic
def set_cell_height(size):
  display(
      Javascript("google.colab.output.setIframeHeight(0, true, {maxHeight: " +
                 str(size) + "})"))

In [5]:
# TensorFlow Decision Forests 버전 확인
print("Found TensorFlow Decision Forests v" + tfdf.__version__)

Found TensorFlow Decision Forests v0.2.7


## 03. 랜덤 포레스트 훈련

In [6]:
# 데이터셋 다운로드
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

# 데이터셋 불러오기
dataset_df = pd.read_csv("/tmp/penguins.csv")

# 상위 3개의 데이터 출력
dataset_df.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007


In [7]:
label = "species"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

dataset_df[label] = dataset_df[label].map(classes.index)

Label classes: ['Adelie', 'Gentoo', 'Chinstrap']


In [8]:
# 데이터세트 분할
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

242 examples in training, 102 examples for testing.


In [9]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [10]:
# 모델 훈련
%set_cell_height 300

model_1 = tfdf.keras.RandomForestModel()

model_1.compile(
    metrics=["accuracy"])

with sys_pipes():
  model_1.fit(x=train_ds)

<IPython.core.display.Javascript object>

Use /tmp/tmpx9xeoyzj as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.613122. Found 242 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpx9xeoyzj/model/ with prefix bd189f22c1744cc6
[INFO abstract_model.cc:1248] Engine "RandomForestGeneric" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:00:00.082022
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [11]:
# 모델 평가
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 1s 708ms/step - loss: 0.0000e+00 - accuracy: 0.9902

loss: 0.0000
accuracy: 0.9902


In [12]:
# 모델 내보내기
model_1.save("/tmp/my_saved_model")

INFO:tensorflow:Assets written to: /tmp/my_saved_model/assets


INFO:tensorflow:Assets written to: /tmp/my_saved_model/assets


In [13]:
# 모델 플로팅
tfdf.model_plotter.plot_model_in_colab(model_1, tree_idx=0, max_depth=3)

In [14]:
# 모델 구조 및 기능 중요도
%set_cell_height 300
model_1.summary()

<IPython.core.display.Javascript object>

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (7):
	bill_depth_mm
	bill_length_mm
	body_mass_g
	flipper_length_mm
	island
	sex
	year

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.           "__LABEL"  3.140619 ################
    2.               "sex"  3.117510 ###############
    3.              "year"  3.115809 ###############
    4.       "body_mass_g"  2.646164 ###########
    5.            "island"  2.157530 #######
    6.     "bill_depth_mm"  1.952792 #####
    7.    "bill_length_mm"  1.399079 
    8. "flipper_length_mm"  1.384412 

Variable Importance: NUM_AS_ROOT:
    1. "flipper_length_mm" 143.000000 ################
    2.    "bill_length_mm" 75.000000 

In [15]:
# 입력 특성
model_1.make_inspector().features()

["bill_depth_mm" (1; #0),
 "bill_length_mm" (1; #1),
 "body_mass_g" (1; #2),
 "flipper_length_mm" (1; #3),
 "island" (4; #4),
 "sex" (4; #5),
 "year" (1; #6)]

In [16]:
# 기능 중요도
model_1.make_inspector().variable_importances()

{'MEAN_MIN_DEPTH': [("__LABEL" (4; #7), 3.1406188256188154),
  ("sex" (4; #5), 3.1175103600103506),
  ("year" (1; #6), 3.1158094220594124),
  ("body_mass_g" (1; #2), 2.646163974913972),
  ("island" (4; #4), 2.157529775779775),
  ("bill_depth_mm" (1; #0), 1.9527923280423283),
  ("bill_length_mm" (1; #1), 1.3990786065786067),
  ("flipper_length_mm" (1; #3), 1.3844115606615612)],
 'NUM_AS_ROOT': [("flipper_length_mm" (1; #3), 143.0),
  ("bill_length_mm" (1; #1), 75.0),
  ("bill_depth_mm" (1; #0), 68.0),
  ("island" (4; #4), 9.0),
  ("body_mass_g" (1; #2), 5.0)],
 'NUM_NODES': [("bill_length_mm" (1; #1), 615.0),
  ("bill_depth_mm" (1; #0), 416.0),
  ("flipper_length_mm" (1; #3), 332.0),
  ("body_mass_g" (1; #2), 286.0),
  ("island" (4; #4), 263.0),
  ("sex" (4; #5), 19.0),
  ("year" (1; #6), 11.0)],
 'SUM_SCORE': [("bill_length_mm" (1; #1), 24993.7682845816),
  ("flipper_length_mm" (1; #3), 21750.989928830415),
  ("bill_depth_mm" (1; #0), 13241.040874174796),
  ("island" (4; #4), 9975.0203

In [17]:
# 모델 자체 평가
model_1.make_inspector().evaluation()

Evaluation(num_examples=242, accuracy=0.9793388429752066, loss=0.07807398846850169, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)

In [18]:
# 훈련 로그 플로팅
%set_cell_height 150
model_1.make_inspector().training_logs()

<IPython.core.display.Javascript object>

[TrainLog(num_trees=1, evaluation=Evaluation(num_examples=90, accuracy=0.8666666666666667, loss=4.80582021077474, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=11, evaluation=Evaluation(num_examples=241, accuracy=0.975103734439834, loss=0.6509367458800557, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=21, evaluation=Evaluation(num_examples=242, accuracy=0.9669421487603306, loss=0.22116267157733932, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=31, evaluation=Evaluation(num_examples=242, accuracy=0.9710743801652892, loss=0.2229844139165376, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=41, evaluation=Evaluation(num_examples=242, accuracy=0.9710743801652892, loss=0.08891751811154618, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)),
 TrainLog(num_trees=51, evaluation=Evaluation(num_examples=242, accuracy=0.9710743801652892, loss=0.09196838698049714, rmse=None

In [19]:
# 다른 학습 알고리즘으로 모델 재훈련
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [20]:
# 도움말 출력하기
help(tfdf.keras.RandomForestModel)
tfdf.keras.RandomForestModel?

Help on class RandomForestModel in module tensorflow_decision_forests.keras:

class RandomForestModel(tensorflow_decision_forests.keras.wrappers.RandomForestModel)
 |  RandomForestModel(*args, **kwargs)
 |  
 |  Random Forest learning algorithm.
 |  
 |  A Random Forest (https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)
 |  is a collection of deep CART decision trees trained independently and without
 |  pruning. Each tree is trained on a random subset of the original training 
 |  dataset (sampled with replacement).
 |  
 |  The algorithm is unique in that it is robust to overfitting, even in extreme
 |  cases e.g. when there is more features than training examples.
 |  
 |  It is probably the most well-known of the Decision Forest training
 |  algorithms.
 |  
 |  Usage example:
 |  
 |  ```python
 |  import tensorflow_decision_forests as tfdf
 |  import pandas as pd
 |  
 |  dataset = pd.read_csv("project/dataset.csv")
 |  tf_dataset = tfdf.keras.pd_dataframe_to_tf_datase

In [21]:
%set_cell_height 300

feature_1 = tfdf.keras.FeatureUsage(name="year", semantic=tfdf.keras.FeatureSemantic.CATEGORICAL)
feature_2 = tfdf.keras.FeatureUsage(name="bill_length_mm")
feature_3 = tfdf.keras.FeatureUsage(name="sex")
all_features = [feature_1, feature_2, feature_3]

model_3 = tfdf.keras.GradientBoostedTreesModel(features=all_features, exclude_non_specified_features=True)
model_3.compile( metrics=["accuracy"])

with sys_pipes():
  model_3.fit(x=train_ds, validation_data=test_ds)

<IPython.core.display.Javascript object>

Use /tmp/tmpm9bfon0e as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.375253. Found 242 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(102, shape=(), dtype=int32)
Validation dataset read in 0:00:00.735919. Found 102 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpm9bfon0e/model/ with prefix b773e6e228e647e4
[INFO abstract_model.cc:1248] Engine "GradientBoostedTreesGeneric" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:00:00.251537
Compiling model...
Model compiled.


In [22]:
# 하이퍼 매개변수
model_6 = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500, growing_strategy="BEST_FIRST_GLOBAL", max_depth=8)
model_6.fit(x=train_ds)

Use /tmp/tmp3vosfo7j as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.170569. Found 242 examples.
Training model...
Model trained in 0:00:00.559350
Compiling model...
Model compiled.


In [23]:
# 정확한 모델 설계하기
model_7 = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE",
    categorical_algorithm="RANDOM",
    )
model_7.fit(x=train_ds)

Use /tmp/tmpqpzlrly9 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.384827. Found 242 examples.
Training model...
Model trained in 0:00:01.026632
Compiling model...
Model compiled.


In [24]:
model_8 = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1")
model_8.fit(x=train_ds)

Resolve hyper-parameter template "benchmark_rank1" to "benchmark_rank1@v1" -> {'growing_strategy': 'BEST_FIRST_GLOBAL', 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}.
Use /tmp/tmp2b81xb28 as temporary training directory
Reading training dataset...


Training dataset read in 0:00:00.159498. Found 242 examples.
Training model...
Model trained in 0:00:00.769458
Compiling model...


Model compiled.


In [25]:
print(tfdf.keras.GradientBoostedTreesModel.predefined_hyperparameters())

[HyperParameterTemplate(name='better_default', version=1, parameters={'growing_strategy': 'BEST_FIRST_GLOBAL'}, description='A configuration that is generally better than the default parameters without being more expensive.'), HyperParameterTemplate(name='benchmark_rank1', version=1, parameters={'growing_strategy': 'BEST_FIRST_GLOBAL', 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}, description='Top ranking hyper-parameters on our benchmark slightly modified to run in reasonable time.')]


In [26]:
# 기능 전처리
%set_cell_height 300

body_mass_g = tf.keras.layers.Input(shape=(1,), name="body_mass_g")
body_mass_kg = body_mass_g / 1000.0

bill_length_mm = tf.keras.layers.Input(shape=(1,), name="bill_length_mm")

raw_inputs = {"body_mass_g": body_mass_g, "bill_length_mm": bill_length_mm}
processed_inputs = {"body_mass_kg": body_mass_kg, "bill_length_mm": bill_length_mm}

preprocessor = tf.keras.Model(inputs=raw_inputs, outputs=processed_inputs)

model_4 = tfdf.keras.RandomForestModel(preprocessing=preprocessor)
model_4.fit(x=train_ds)

model_4.summary()

<IPython.core.display.Javascript object>

Use /tmp/tmplz3i_9u7 as temporary training directory
Reading training dataset...


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['island', 'bill_depth_mm', 'flipper_length_mm', 'sex', 'year'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Training dataset read in 0:00:00.467635. Found 242 examples.
Training model...
Model trained in 0:00:00.135764
Compiling model...


Model compiled.


Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          {'body_mass_kg': (None,   0         
                             1),                                 
                              'bill_length_mm': (None            
                             , 1)}                               
                                                                 
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (2):
	bill_length_mm
	body_mass_kg

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.        "__LABEL"  3.769471 ################
    2.   "body_mass_kg"  1.196381 ####
    3. "bill_length_mm"  0.055515 

Variable Importance: NUM_AS_ROOT:
    1. "bill_length_mm" 284.000000 ################
    2.  

In [27]:
def g_to_kg(x):
  return x / 1000

feature_columns = [
    tf.feature_column.numeric_column("body_mass_g", normalizer_fn=g_to_kg),
    tf.feature_column.numeric_column("bill_length_mm"),
]

preprocessing = tf.keras.layers.DenseFeatures(feature_columns)

model_5 = tfdf.keras.RandomForestModel(preprocessing=preprocessing)
model_5.compile(metrics=["accuracy"])
model_5.fit(x=train_ds)

Use /tmp/tmp_s19_e48 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.300989. Found 242 examples.
Training model...
Model trained in 0:00:00.102814
Compiling model...
Model compiled.


## 04. 회귀 모델 학습

In [28]:
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/abalone_raw.csv -O /tmp/abalone.csv

dataset_df = pd.read_csv("/tmp/abalone.csv")
print(dataset_df.head(3))

  Type  LongestShell  Diameter  Height  WholeWeight  ShuckedWeight  \
0    M         0.455     0.365   0.095       0.5140         0.2245   
1    M         0.350     0.265   0.090       0.2255         0.0995   
2    F         0.530     0.420   0.135       0.6770         0.2565   

   VisceraWeight  ShellWeight  Rings  
0         0.1010         0.15     15  
1         0.0485         0.07      7  
2         0.1415         0.21      9  


In [29]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

label = "Rings"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

2925 examples in training, 1252 examples for testing.


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [30]:
%set_cell_height 300

model_7 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

model_7.compile(metrics=["mse"])

# 모델 훈련
with sys_pipes():
  model_7.fit(x=train_ds)

<IPython.core.display.Javascript object>

Use /tmp/tmpu_pz_tk5 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.176100. Found 2925 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpu_pz_tk5/model/ with prefix bfce140687b94922
[INFO abstract_model.cc:1248] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:00:02.678855
Compiling model...
Model compiled.


In [31]:
evaluation = model_7.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")

3/3 [==============================] - 0s 80ms/step - loss: 0.0000e+00 - mse: 1.8886
{'loss': 0.0, 'mse': 1.888583779335022}

MSE: 1.888583779335022
RMSE: 1.3742575374852495


## 05. 순위 모델 학습

In [32]:
%set_cell_height 200

archive_path = tf.keras.utils.get_file("letor.zip",
  "https://download.microsoft.com/download/E/7/E/E7EABEF1-4C7B-4E31-ACE5-73927950ED5E/Letor.zip",
  extract=True)

raw_dataset_path = os.path.join(os.path.dirname(archive_path),"OHSUMED/Data/All/OHSUMED.txt")

<IPython.core.display.Javascript object>

61824018/61824018 [==============================] - 2s 0us/step


In [33]:
def convert_libsvm_to_csv(src_path, dst_path):
  dst_handle = open(dst_path, "w")
  first_line = True
  for src_line in open(src_path,"r"):
    items = src_line.split(" ")[:-3]
    relevance = items[0]
    group = items[1].split(":")[1]
    features = [ item.split(":") for item in items[2:]]

    if first_line:
      dst_handle.write("relevance,group," + ",".join(["f_" + feature[0] for feature in features]) + "\n")
      first_line = False
    dst_handle.write(relevance + ",g_" + group + "," + (",".join([feature[1] for feature in features])) + "\n")
  dst_handle.close()

# 데이터세트 변환
csv_dataset_path="/tmp/ohsumed.csv"
convert_libsvm_to_csv(raw_dataset_path, csv_dataset_path)

# 데이터세트 불러오기
dataset_df = pd.read_csv(csv_dataset_path)

# 상위 3개 출력.
dataset_df.head(3)

,relevance,group,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25
0,2,g_1,3.0,2.079442,0.272727,0.261034,37.330565,11.431241,37.29975,1.138657,...,9.340024,24.808785,0.393091,57.416517,3.294893,25.0231,3.219799,-3.87098,-3.90273,-3.87512
1,0,g_1,3.0,2.079442,0.428571,0.400594,37.330565,11.431241,37.29975,1.814480,...,9.340024,24.808785,0.349205,43.240626,2.654724,23.4903,3.156588,-3.96838,-4.00865,-3.98670
2,2,g_1,0.0,0.000000,0.000000,0.000000,37.330565,11.431241,37.29975,0.000000,...,9.340024,24.808785,0.240319,25.816989,1.551342,15.8650,2.764115,-4.28166,-4.33313,-4.44161


In [34]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

train_ds_pd.head(3)

11381 examples in training, 4759 examples for testing.


,relevance,group,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25
0,2,g_1,3.0,2.079442,0.272727,0.261034,37.330565,11.431241,37.29975,1.138657,...,9.340024,24.808785,0.393091,57.416517,3.294893,25.0231,3.219799,-3.87098,-3.90273,-3.87512
1,0,g_1,3.0,2.079442,0.428571,0.400594,37.330565,11.431241,37.29975,1.814480,...,9.340024,24.808785,0.349205,43.240626,2.654724,23.4903,3.156588,-3.96838,-4.00865,-3.98670
2,2,g_1,0.0,0.000000,0.000000,0.000000,37.330565,11.431241,37.29975,0.000000,...,9.340024,24.808785,0.240319,25.816989,1.551342,15.8650,2.764115,-4.28166,-4.33313,-4.44161


In [35]:
relevance = "relevance"

ranking_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=relevance, task=tfdf.keras.Task.RANKING)
ranking_test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=relevance, task=tfdf.keras.Task.RANKING)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [36]:
%set_cell_height 400

model_8 = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.RANKING,
    ranking_group="group",
    num_trees=50)

with sys_pipes():
  model_8.fit(x=ranking_train_ds)

<IPython.core.display.Javascript object>

Use /tmp/tmpjcdh061f as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.748533. Found 11381 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpjcdh061f/model/ with prefix 4390be32d1714b83


Model trained in 0:00:02.588259


[INFO abstract_model.cc:1248] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO kernel.cc:1022] Use fast generic engine


Compiling model...
Model compiled.


In [37]:
%set_cell_height 400

model_8.summary()

<IPython.core.display.Javascript object>

Model: "gradient_boosted_trees_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: RANKING
Label: "__LABEL"
Rank group: "__RANK_GROUP"

Input Features (25):
	f_1
	f_10
	f_11
	f_12
	f_13
	f_14
	f_15
	f_16
	f_17
	f_18
	f_19
	f_2
	f_20
	f_21
	f_22
	f_23
	f_24
	f_25
	f_3
	f_4
	f_5
	f_6
	f_7
	f_8
	f_9

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1. "__RANK_GROUP"  4.354192 ################
    2.          "f_1"  4.354192 ################
    3.      "__LABEL"  4.354192 ################
    4.          "f_5"  4.349919 ###############
    5.          "f_6"  4.327043 ###############
    6.         "f_17"  4.322493 ###############
    7.         "f_12"  4.316293 ###############
    8.         "f_24"  4.311271 ###############
    9.       